## **TSFRESH Feature Extraction**

In this notebook, a template pipeline for clustering from the features extracted using the TSFRESH python package is presented. For demonstration purposes, we have used the time-series data for the calibration period ([-15, -3] from sample detect time) without standardization and without noise. reduction. 

Note that this same pipeline has been applied to the following windows: 

* calibration window ([-15, -3] from sample detect time) - without standardization
* calibration window ([-15, -3] from sample detect time) - standardized before windowed
* post window ([12, 16] from sample detect time) - standardized before windowed
* sample window ([32, 35] from sample detect time) - standardized before windowed
* whole timeseries from [-30, 40] (w.r.t sample detect time) - normalized and smoothed (using convolution with a bartlett window of len = 50)

This pipeline did not allow us to cluster the ECDs. The PCA visualizations helped us notice just how similar the ECD readings were from the unsuccessful readings.

1. Extract features using the `feature_matrix` function
2. Apply PCA to these features for dimension reduction
3. Use hierarchical cluster or Gaussian Mixture Modelling to find clusters and subclusters by applying a multi-step approach.
    
Throughout this notebook, visualizations are used in order to inform the user on what steps should be taken next (i.e. tuning the parameter for the number of clusters, deciding whether there seems to be clusters forming etc.), which means the process is quite subjective. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly                                          ## Package necessary for 3D represntation of the PCA components
import plotly.graph_objs as go
import altair as alt

from tsfresh import extract_features, select_features  ## Package necessary for extracting the features
from tsfresh.utilities.dataframe_functions import impute

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as shc

pd.options.mode.chained_assignment = None  # default='warn'
alt.data_transformers.disable_max_rows()

### **1- FEATURE EXTRACTION**

In [ ]:
def feature_matrix(un_data, ecd_data, scale = True):
    """Creates a feature matrix from the raw timeseries and stores them in a dataframe.
    Args:
        un_data (pandas data frame): Time series for unsuccessful readings stored in rows of dataframe, with first column being the test ID.
        ecd_data (pandas data frame): Time series for ECD readings stored in rows of dataframe, with first column being the test ID.
        scale (bool, optional): If True, the feature matrix is standardized (preferred when clustering).
    Returns:
        A new pandas data frame containing various features (columns) for each reading/timeseries (rows).
    """
    # Adding labels to our timeseries
    un_data['label'] = False
    ecd_data['label'] = True
    
    # Making one dataset with unsuccessful and ECD
    data = pd.concat([un_data, ecd_data]).reset_index(drop = True)
    
    # Changing format of data to fit requirements of `extract_features` function
    melt_data = pd.melt(data, id_vars = ['TestId', 'label'], var_name = 'time')
    melt_data['time'] = pd.to_numeric(melt_data['time'])
    melt_data = melt_data.sort_values(by = ['TestId', 'time']).reset_index(drop = True)
   
    # We use squeeze to go from dataframe to series because the `select_features` function needs it to be
    y = data[['TestId', 'label']].sort_values(by = 'TestId').set_index('TestId')
    y_series = y.squeeze()
    
    # Now we want to extract features seperately for all ids
    extracted_features = extract_features(melt_data, column_id="TestId", column_sort="time", column_value = 'value')
    
    # Remove all features containing NaN values (which were created because could not be calculated on the time series, i.e. stat too low)
    extracted_features.dropna(axis='columns', inplace = True)
    
    # Select the relevant features
    impute(extracted_features)
    features_filtered = select_features(extracted_features, y_series)
    
    # Scale the feature matrix
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features_filtered)
    
    print(f"The total number of features created is: {len(extracted_features.columns)}")
    print(f"The total number of relevant features is: {len(features_filtered.columns)}")
    
    #Convert to data frame with test ids as index and appropriate column labels
    scaled_features  = pd.DataFrame(scaled_features, columns = features_filtered.columns).set_index(features_filtered.index)
    
    # Making TesId a column instead of the index
    scaled_features = y.join(scaled_features, on='TestId').reset_index(level = 0)
    features_filtered = y.join(features_filtered, on = 'TestId').reset_index(level = 0)
    
    if scale == True:
        return scaled_features
    else:
        return features_filtered

In [ ]:
# Importing the timeseries data
ecd_cal = pd.read_csv("/Users/neethug/Desktop/Neethu/Course/DATA599/Project/Data/data/ecd_cal.csv")
un_cal = pd.read_csv("/Users/neethug/Desktop/Neethu/Course/DATA599/Project/Data/data/un_cal.csv")
syn_cal = pd.read_csv("/Users/neethug/Desktop/Neethu/Course/DATA599/Project/Data/data/syn_cal.csv")

In [ ]:
# Concatenate the synthetic ECDs with the wild ECDs
ecd_cal_tot = pd.concat([ecd_cal, syn_cal])

In [ ]:
# Removing outliers (TestId 9610647, 9610462) --> Identified outliers because they were on a completely different scale compared to all the other readings
ecd_cal_tot = ecd_cal_tot[~ecd_cal_tot['TestId'].isin([9610647, 9610462])]

In [ ]:
# Extracting the features
feat_cal = feature_matrix(un_data = un_cal, ecd_data = ecd_cal_tot, scale = True)

### **2- DIMENSION REDUCTION WITH PCA**

In [ ]:
# Applying PCA on all the scaled relevant features
pca = PCA(n_components = 0.95) # keeping 95% of the cumulative variance explained
principalComponents = pca.fit_transform(feat_cal.drop(columns = ['label', 'TestId']))
principalDf = pd.DataFrame(principalComponents, columns = [f"PC_{i}" for i in range(1, len(pca.components_) + 1)])
principalDf

In [ ]:
# Scree plot to see how many components to keep
plt.plot(range(1, len(pca.components_) + 1), pca.explained_variance_ratio_, 'ro-', linewidth=2)
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Proportion of Variance Explained')
plt.show()

Here, we see that the elbow of the screeplot is at around 5 principal components. This being said, if we only keep the first 5 components, they would account for a cumulative proportion of variance of only ~63 % of our data (as seen below). 

In [ ]:
# Finding how much variation is explained by each component
cumulative_sum = np.cumsum(pca.explained_variance_ratio_) 
print(f"Proportion of Variance Explained : {np.round(pca.explained_variance_ratio_,2)}")   
print("Cumulative Prop. Variance Explained: ", cumulative_sum)

In [ ]:
# The matrix of variable loadings (i.e., matrix whose columns contain the eigenvectors)
# The eigenvectos provide the coefficients for the linear combo
rotation = pd.DataFrame(pca.components_, columns = feat_cal.columns[2:]).T 
rotation.columns = [f"PC_{i}" for i in range(1, len(pca.components_) + 1)]
rotation

In [ ]:
# The top 5 features that load on the first PC
rotation.sort_values(by=['PC_1'], key = abs, ascending = False).head(5)[['PC_1']]

In [ ]:
# The top 5 features that load on the second PC
rotation.sort_values(by=['PC_2'], key = abs, ascending = False).head(5)[['PC_2']]

For detailed information on these features and what they represent, the tsfresh documentation can be consulted here: [link](https://tsfresh.readthedocs.io/en/latest/text/list_of_features.html)

In [ ]:
# Adding labels (True: For ECD, False: For unsucessful readings that arent ECD errors) to the scores for visualizations purposes.
y = feat_cal[['TestId', 'label']]
principalDfLabel = pd.concat([y, principalDf, ], axis = 1)
principalDfLabel

#### **Visualizations**

In [ ]:
alt.data_transformers.disable_max_rows()
alt.Chart(principalDfLabel, title = 'Unstandardized Calibration Window').mark_point().encode(
    alt.X('PC_1'),
    alt.Y('PC_2'),
    alt.Color('label'))

In [ ]:
alt.Chart(principalDfLabel).mark_circle().encode(
    alt.X(alt.repeat("column"), type='quantitative'),
    alt.Y(alt.repeat("row"), type='quantitative'),
    alt.Color('label')
).properties(
    width=150,
    height=150
).repeat(
    row = [f"PC_{i}" for i in range(1, len(pca.components_) + 1)][0:5],
    column = [f"PC_{i}" for i in range(1, len(pca.components_) + 1)][0:5])

In [ ]:
# 3D representation
plotly.offline.init_notebook_mode()

# Changing True/False label to a color because somehow passing the label column to the `color` marker does not work.
color = []
for i in range(len(principalDfLabel)):
    if principalDfLabel['label'][i] == True:
        color.append("orange")
    else:
        color.append("blue")

# Configure the trace.
trace = go.Scatter3d(
    x = principalDfLabel['PC_1'],  
    y = principalDfLabel['PC_2'], 
    z = principalDfLabel['PC_3'],
    mode = 'markers',
    marker ={
        'size': 5,
        'opacity': 0.8,
        'color':color
    }
)

# Configure the layout.
layout = go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0}
)

data = [trace]

plot_figure = go.Figure(data = data, layout = layout)

# Render the plot.
plotly.offline.iplot(plot_figure)

### **3- CLUSTERING USING THE COMPONENTS FROM PCA**

#### **3.1) Using Gaussian Mixture Modelling**

In [ ]:
# Finding the number of unique return codes in our data to help us set an initial number of cluster (Here we have 31)
# ecd_pred = pd.read_csv("../../../../data/RawDataPredictors/New/ECDContact.csv")
# syn_pred = pd.read_csv("../../../../data/RawDataPredictors/New/SyntheticECD.csv") 
# un_pred = pd.read_csv("../../../../data/RawDataPredictors/New/Unsuccessful.csv")

# num_returncodes = len(np.unique(np.concatenate((np.unique(un_pred['ReturnCode']),np.unique(ecd_pred['ReturnCode'])))))
# num_returncodes

In this pipeline we applied Gaussian Mixture Modelling as the first clustering algorithm. Other clustering algorithms such as hierarchical clustering could have been used instead with various types of links (they have been tried, and yielded similar results). The code for hierarchical clustering is provided but commented out for the purpose of this demonstration. 

In [ ]:
# plt.figure(figsize=(5, 5))  
# plt.title("Dendrograms")  
# dend = shc.dendrogram(shc.linkage(principalDf, method='ward'))

In [ ]:
# hier_clustering = AgglomerativeClustering(n_clusters = 2, affinity = 'euclidean', linkage = 'ward')
# hier_clustering.fit_predict(principalDf)
# y['clusters'] = hier_clustering.labels_

Back to Gaussian Mixture Modelling:

In [ ]:
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components = 31)
gmm.fit(principalDf)

#predictions from gmm
clusters = gmm.predict(principalDf)

y['clusters'] = clusters

In [ ]:
y.head()

In [ ]:
# Getting an idea of how the clusters are distributed
num_clusters = 31

for i in range(num_clusters):
    print(f"The proportion of ECD errors in cluster {i} is: {np.round(len(y[(y['label'] == True) & (y['clusters'] == i)])/len(y[y['clusters'] == i]),4)}")
    print(f"Out of all the {len(ecd_cal_tot)} ECDs, {len(y[(y['label'] == True) & (y['clusters'] == i)])} are in cluster {i}, which represents {np.round((len(y[(y['label'] == True) & (y['clusters'] == i)])/len(ecd_cal_tot))*100, 2)}% of all the total ECDs") 
    print(f"The total number of readings in cluster {i} is {len(y[y['clusters'] == i])}")
    print("")

Ideally, we would want a cluster that contains a high proportion of ECDs (i.e a proportion of 50% would mean that the cluster contains an equal amount of ecd errors and unsuccessful readings) as well as a cluster that contains most of the ECDs. We could then try applying another clustering algorithm only on that cluster in order to further extract the ECDs from the unsuccessful readings.

We have decided to take a closer look at the clusters that are made up of at least 20% ECDs and include at least 20% of all the ECDs (i.e 20% of 324 ~ 65 ECDs). These threshold can be changed in the following cell as desired.

In [ ]:
important_clusters = []
for i in range(num_clusters):
    prop_ecd = (len(y[(y['label'] == True) & (y['clusters'] == i)])/len(ecd_cal_tot)) # (number of ecd in cluster i)/(total number of ecd)
    prop_ecd_cluster = len(y[(y['label'] == True) & (y['clusters'] == i)])/len(y[y['clusters'] == i]) # (number of ecd in cluster i)/(total number of readings in cluster i)
    if (prop_ecd > 0.2) & (prop_ecd_cluster > 0.2):
        important_clusters.append(i)
important_clusters

##### **3.1.1) Forming subclusters**

In [ ]:
subclusterids = y[y['clusters'].isin(important_clusters)][['TestId', 'label', 'clusters']].reset_index(drop = True)
subclusterPC = principalDfLabel[principalDfLabel['TestId'].isin(subclusterids['TestId'])].reset_index(drop = True)

# Merging the cluster for visualization purposes
subclusterPC = subclusterPC.merge(subclusterids[['TestId', 'clusters']], on = 'TestId')
subclusterPC.head()

In [ ]:
subclusterY = y[y['TestId'].isin(subclusterids['TestId'])].reset_index(drop = True)

##### **3.1.2) Visualizing the components only for the subclusters**

In [ ]:
alt.data_transformers.disable_max_rows()
alt.Chart(subclusterPC, title = 'Unstandardized Calibration Window for the sub clusters').mark_point().encode(
    alt.X('PC_1'),
    alt.Y('PC_2'),
    alt.Color('label'))

From the above plot, we can easily imagine why it would be hard to cluster out the ECDs (orange) from the unsucessful readings (blue). Lets try anyways using hierarchical clustering.

#### **3.2) Subclustering using hierarchical clustering**

Once again, any type of clustering could have been used here instead of hierarchical clustering. We have also tried using Gaussian Mixture Modelling, BIRCH, DBSCAN without obtaining significantly better results.

In [ ]:
plt.figure(figsize=(5, 5))  
plt.title("Dendrograms")  
dend = shc.dendrogram(shc.linkage(subclusterPC.drop(['TestId', 'label', 'clusters'], axis = 1), method='ward'))

From the dendrogram, we see that 2 clusters would be optimal

In [ ]:
hier_clustering = AgglomerativeClustering(n_clusters = 2, affinity = 'euclidean', linkage = 'ward')
hier_clustering.fit_predict(subclusterPC.drop(['TestId', 'label', 'clusters'], axis = 1))

subclusterY['sub_clusters'] = hier_clustering.labels_

In [ ]:
len(subclusterY[subclusterY['label'] == True])

In [ ]:
num_clusters = 2

for i in range(num_clusters):
    print(f"The proportion of ECD errors in cluster {i} is: {np.round(len(subclusterY[(subclusterY['label'] == True) & (subclusterY['sub_clusters'] == i)])/len(subclusterY[subclusterY['sub_clusters'] == i]),4)}")
    print(f"Out of all the {len(subclusterY[subclusterY['label'] == True])} ECDs in cluster(s) {important_clusters}, {len(subclusterY[(subclusterY['label'] == True) & (subclusterY['sub_clusters'] == i)])} are in cluster {i}, which represents {np.round((len(subclusterY[(subclusterY['label'] == True) & (subclusterY['sub_clusters'] == i)])/len(subclusterY[subclusterY['label'] == True]))*100, 2)}%") 
    print(f"The total number of readings in subcluster {i} is {len(subclusterY[subclusterY['sub_clusters'] == i])}")
    print("")

As we can see from the output above, the size of the 2 clusters formed are very similar to one another and so is the proportion of ECDs in each cluster. We were thus not able to successfully extract the ECDs from this pipeline.

Further investigation using other clustering algorithms and/or different parameters and/or applied to a different time window could yield better (or worse) results.